In [1]:
from faceio import get_date_directories

In [2]:
video_by_day = get_date_directories('bili_video', RATIO = 1)

In [4]:
len(video_by_day)

6

In [3]:
import glob, math
import numpy as np
from tqdm import tqdm, trange
import cv2
import dlib
import os
i = 0
lastFrame = None


detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')


def getLandmarks(img):
    dets = detector(img, 1)
    for k, d in enumerate(dets):
        shape = predictor(img, d)
        currPoints = np.array([shape.part(i) for i in range(48, 68)])
        return currPoints
    dlib.hit_enter_to_continue()

def rotate(origin, point, angle):
    """
    Rotate a point counterclockwise by a given angle around a given origin.

    The angle should be given in radians.
    """
    ox, oy = origin
    px, py = point

    qx = ox + math.cos(angle) * (px - ox) - math.sin(angle) * (py - oy)
    qy = oy + math.sin(angle) * (px - ox) + math.cos(angle) * (py - oy)
    return qx, qy

def read_video(file, num):
    videos = []
    cap = cv2.VideoCapture(file)
    n_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    if fps != 25:
        print('[!] WARN: Framerate inconsistent.')
    dur = n_frames/fps
    
    for i in range(num):
        _, frame = cap.read()
        if frame is None:
            frame = lastFrame
        else:
            lastFrame = frame
        videos.append(frame)
    return videos

for day in trange(4, 5):
    if os.path.exists(video_by_day[day]['target']):
        toBeConcat = []
        # the outer loop is we enumerate each video, we need to concatenate all of them
        for index, (file, num) in enumerate(video_by_day[day]['sources']):
            # videos is images
            frames = read_video(file, num)
            # this loop is to enumerate all the pictures
            for img_num, img in enumerate(frames):
                try:
                    points = getLandmarks(img) # get landmarks return a set of points for a given image
                    # this loop is to enumerate all the pictures
                    origin_x = (points[0].x + points[6].x)/2
                    origin_y = (points[0].y + points[6].y)/2
                    origin = (origin_x, origin_y)
                    
                    deltaY = points[6].y - points[0].y
                    deltaX = points[6].x - points[0].x
                    angle = -1 * math.atan2(deltaY, deltaX)
                    new_points = []

                    scaleFactor = 1
                    for i, _point in enumerate(points):
                        newpoint = list(rotate(origin, (_point.x, _point.y), angle))
                        if i == 0:
                            scaleFactor = -1/(newpoint[0] - origin_x)
                        newpoint[0] = (newpoint[0] - origin_x)*scaleFactor
                        newpoint[1] = (newpoint[1] - origin_y)*scaleFactor
                        new_points.append(newpoint)
                        # print(newpoint) 
                    new_points = np.concatenate(new_points).reshape(20,2)
                    toBeConcat.append(new_points)
                except:
                    print(img_num, img, file)

            output = np.stack(toBeConcat)
            np.save(video_by_day[day]['target'], output)


    

  0%|          | 0/1 [00:00<?, ?it/s]

187 [[[ 0  0  0]
  [ 0  0  0]
  [ 0  0  0]
  ...
  [ 0  0  0]
  [ 0  0  0]
  [ 0  0  0]]

 [[ 0  0  0]
  [ 0  0  0]
  [ 0  0  0]
  ...
  [ 0  0  0]
  [ 0  0  0]
  [ 0  0  0]]

 [[ 0  0  0]
  [ 0  0  0]
  [ 0  0  0]
  ...
  [ 0  0  0]
  [ 0  0  0]
  [ 0  0  0]]

 ...

 [[ 9  0 47]
  [17  0 55]
  [18  0 56]
  ...
  [14  9 15]
  [ 6  1  7]
  [ 2  0  3]]

 [[ 9  0 47]
  [18  0 56]
  [18  0 56]
  ...
  [14  9 15]
  [ 6  1  7]
  [ 2  0  3]]

 [[ 9  0 47]
  [18  0 56]
  [18  0 56]
  ...
  [13  8 14]
  [ 7  2  8]
  [ 2  0  3]]] /home/yunyangli/CMLR/bili_video/M/LuoXiang_M_Av330752427/LuoXiang_M_Av330752427_1min36s.mp4


100%|██████████| 1/1 [07:31<00:00, 451.07s/it]


In [12]:
video_by_day[4]

{'sources': [('/home/yunyangli/CMLR/bili_video/M/LuoXiang_M_Av330752427/LuoXiang_M_Av330752427_1min36s.mp4',
   231),
  ('/home/yunyangli/CMLR/bili_video/M/LuoXiang_M_Av330752427/LuoXiang_M_Av330752427_1min59s.mp4',
   193),
  ('/home/yunyangli/CMLR/bili_video/M/LuoXiang_M_Av330752427/LuoXiang_M_av970352315_begin_10s.mp4',
   275),
  ('/home/yunyangli/CMLR/bili_video/M/LuoXiang_M_Av330752427/LuoXiang_M_av970352315_begin_1min08s.mp4',
   196),
  ('/home/yunyangli/CMLR/bili_video/M/LuoXiang_M_Av330752427/LuoXiang_M_av970352315_begin_1min15s.mp4',
   251),
  ('/home/yunyangli/CMLR/bili_video/M/LuoXiang_M_Av330752427/LuoXiang_M_av970352315_begin_1min26s.mp4',
   203),
  ('/home/yunyangli/CMLR/bili_video/M/LuoXiang_M_Av330752427/LuoXiang_M_av970352315_begin_22s.mp4',
   148),
  ('/home/yunyangli/CMLR/bili_video/M/LuoXiang_M_Av330752427/LuoXiang_M_av970352315_begin_28s.mp4',
   200),
  ('/home/yunyangli/CMLR/bili_video/M/LuoXiang_M_Av330752427/LuoXiang_M_av970352315_begin_36s.mp4',
   125),
